In [2]:
# Import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
pd.set_option('max_colwidth', 400)

In [3]:
# Importing Data Sets 
demographics_df = pd.read_csv('./Resources/NYC_Demographics.csv')
evictions_df = pd.read_csv('./Resources/NYC_Evictions.csv')

In [4]:
# Create a filtered dataframe from specific columns
demographics_cols = ["JURISDICTION NAME", "COUNT PARTICIPANTS", "PERCENT FEMALE", "PERCENT MALE", 
                     "PERCENT PACIFIC ISLANDER", "PERCENT HISPANIC LATINO", "PERCENT AMERICAN INDIAN", 
                     "PERCENT ASIAN NON HISPANIC", "PERCENT WHITE NON HISPANIC", "PERCENT BLACK NON HISPANIC", 
                     "PERCENT OTHER ETHNICITY", "PERCENT ETHNICITY UNKNOWN", "PERCENT PERMANENT RESIDENT ALIEN", 
                     "PERCENT US CITIZEN", "PERCENT OTHER CITIZEN STATUS", "PERCENT RECEIVES PUBLIC ASSISTANCE", 
                     "PERCENT NRECEIVES PUBLIC ASSISTANCE"]
demographics_filtered_df = demographics_df[demographics_cols]

# Rename the column headers
demographics_filtered_df = demographics_filtered_df.rename(columns={"JURISDICTION NAME": "ZIPCODE"})

# Clean the data by dropping duplicates and dropping zipcodes with no participants
demographics_filtered_df.drop_duplicates("ZIPCODE", inplace=True)
demographics_filtered = demographics_filtered_df.loc[demographics_filtered_df["COUNT PARTICIPANTS"] != 0]


demographics_filtered

,ZIPCODE,COUNT PARTICIPANTS,PERCENT FEMALE,PERCENT MALE,PERCENT PACIFIC ISLANDER,PERCENT HISPANIC LATINO,PERCENT AMERICAN INDIAN,PERCENT ASIAN NON HISPANIC,PERCENT WHITE NON HISPANIC,PERCENT BLACK NON HISPANIC,PERCENT OTHER ETHNICITY,PERCENT ETHNICITY UNKNOWN,PERCENT PERMANENT RESIDENT ALIEN,PERCENT US CITIZEN,PERCENT OTHER CITIZEN STATUS,PERCENT RECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE
0,10001,44,0.50,0.50,0.0,0.36,0.00,0.07,0.02,0.48,0.07,0.00,0.05,0.95,0.0,0.45,0.55
1,10002,35,0.54,0.46,0.0,0.03,0.00,0.80,0.17,0.00,0.00,0.00,0.06,0.94,0.0,0.06,0.94
2,10003,1,1.00,0.00,0.0,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.0,0.00,1.00
4,10005,2,1.00,0.00,0.0,0.00,0.00,0.50,0.00,0.50,0.00,0.00,0.50,0.50,0.0,0.00,1.00
5,10006,6,0.33,0.67,0.0,0.33,0.00,0.00,0.17,0.50,0.00,0.00,0.00,1.00,0.0,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,12783,201,0.33,0.67,0.0,0.00,0.01,0.00,0.97,0.00,0.01,0.00,0.02,0.98,0.0,0.38,0.62
230,12786,4,0.75,0.25,0.0,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,1.00,0.0,0.25,0.75
231,12788,83,0.47,0.53,0.0,0.00,0.00,0.00,0.98,0.00,0.02,0.00,0.00,1.00,0.0,0.42,0.58
232,12789,272,0.42,0.58,0.0,0.00,0.00,0.00,0.96,0.00,0.02,0.01,0.00,1.00,0.0,0.26,0.74


In [5]:
# Convert Executed Date to DateTime
evictions_df["Executed Date"]= pd.to_datetime(evictions_df["Executed Date"])

# Create a filtered dataframe from specific columns and dates in 
evictions_cols = ["Eviction Zip", "Executed Date", "Residential/Commercial"]
evictions_filtered_df = evictions_df[evictions_cols]

evictions_filtered = evictions_filtered_df.loc[evictions_filtered_df["Residential/Commercial"] == "Residential"]

evictions_filtered

,Eviction Zip,Executed Date,Residential/Commercial
0,10456,2018-02-26,Residential
1,10466,2022-11-16,Residential
2,10467,2017-09-29,Residential
3,11213,2018-07-12,Residential
4,10470,2019-10-24,Residential
...,...,...,...
72483,11368,2019-10-28,Residential
72484,11423,2019-08-06,Residential
72485,10472,2018-07-13,Residential
72486,11203,2019-12-04,Residential


In [6]:
# Combine the data into a single dataset, left merging the demographics onto the evictions dataframe.  
combined_df = pd.merge(evictions_filtered, demographics_filtered, left_on="Eviction Zip", right_on="ZIPCODE", how='inner')
combined_df#.count()

,Eviction Zip,Executed Date,Residential/Commercial,ZIPCODE,COUNT PARTICIPANTS,PERCENT FEMALE,PERCENT MALE,PERCENT PACIFIC ISLANDER,PERCENT HISPANIC LATINO,PERCENT AMERICAN INDIAN,PERCENT ASIAN NON HISPANIC,PERCENT WHITE NON HISPANIC,PERCENT BLACK NON HISPANIC,PERCENT OTHER ETHNICITY,PERCENT ETHNICITY UNKNOWN,PERCENT PERMANENT RESIDENT ALIEN,PERCENT US CITIZEN,PERCENT OTHER CITIZEN STATUS,PERCENT RECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE
0,10456,2018-02-26,Residential,10456,5,0.60,0.40,0.0,0.60,0.2,0.00,0.0,0.00,0.2,0.0,0.0,1.0,0.0,0.20,0.80
1,10456,2019-07-15,Residential,10456,5,0.60,0.40,0.0,0.60,0.2,0.00,0.0,0.00,0.2,0.0,0.0,1.0,0.0,0.20,0.80
2,10456,2017-01-04,Residential,10456,5,0.60,0.40,0.0,0.60,0.2,0.00,0.0,0.00,0.2,0.0,0.0,1.0,0.0,0.20,0.80
3,10456,2018-10-01,Residential,10456,5,0.60,0.40,0.0,0.60,0.2,0.00,0.0,0.00,0.2,0.0,0.0,1.0,0.0,0.20,0.80
4,10456,2017-09-05,Residential,10456,5,0.60,0.40,0.0,0.60,0.2,0.00,0.0,0.00,0.2,0.0,0.0,1.0,0.0,0.20,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42032,10464,2017-12-01,Residential,10464,15,0.47,0.53,0.0,0.47,0.0,0.07,0.0,0.47,0.0,0.0,0.0,1.0,0.0,0.47,0.53
42033,10464,2020-01-24,Residential,10464,15,0.47,0.53,0.0,0.47,0.0,0.07,0.0,0.47,0.0,0.0,0.0,1.0,0.0,0.47,0.53
42034,10464,2017-11-21,Residential,10464,15,0.47,0.53,0.0,0.47,0.0,0.07,0.0,0.47,0.0,0.0,0.0,1.0,0.0,0.47,0.53
42035,10464,2017-03-17,Residential,10464,15,0.47,0.53,0.0,0.47,0.0,0.07,0.0,0.47,0.0,0.0,0.0,1.0,0.0,0.47,0.53


In [12]:
zipcode_evictions = combined_df.groupby('Eviction Zip')['ZIPCODE'].count()
zipcode_evictions

Eviction Zip
10001    169
10002    267
10003    123
10005     55
10006     23
        ... 
11420    289
11427    109
11432    504
11434    618
11691    894
Name: ZIPCODE, Length: 80, dtype: int64